In [2]:
x = {
    "a": 0.12,
    "b": 0.34,
    "c": 0.56,
    "d": 0.78,
    "e": 0.90
}

max(x, key=x.get)

'e'

In [1]:
import sys

sys.path.append("/Users/simonprivat/Workspace/Projects/emo-consens-bot")

from bot_system.pepper_gpt import PepperGPT

prompter = PepperGPT(True, True)

try:
    while True:
        pass
except KeyboardInterrupt:
    print("-------------- Disposing --------------")
    prompter.dispose()
    print("-------------- Disposed --------------")

PepperGPT initialized
Detecting speech... 1.00
No face detected
Speech ended... 0.82
Speech duration: 1.66
Hallo, wie geht es dir?. Ich bin kein chatbot. Ich bin ein Mensch.
ChatAgent Answered: 
Hallo, wie geht es dir?. Ich bin kein chatbot. Ich bin ein Mensch.
Speech too short... 0.00
Detecting speech... 0.96
Speech ended... 0.82
Speech duration: 2.05
Hallo, wie geht es dir?. Ich bin kein chatbot. Ich bin ein Mensch.
ChatAgent Answered: 
Hallo, wie geht es dir?. Ich bin kein chatbot. Ich bin ein Mensch.
Speech too short... 0.00
Detecting speech... 0.93
Speech ended... 0.79
Speech duration: 1.92
Hallo, wie geht es dir?. Ich bin kein chatbot. Ich bin ein Mensch.
ChatAgent Answered: 
Hallo, wie geht es dir?. Ich bin kein chatbot. Ich bin ein Mensch.
Speech too short... 0.00
Detecting speech... 0.93
Speech ended... 0.75
Speech duration: 1.54
Hallo, wie geht es dir?. Ich bin kein chatbot. Ich bin ein Mensch.
ChatAgent Answered: 
Hallo, wie geht es dir?. Ich bin kein chatbot. Ich bin ein Me

In [6]:
from deepface import DeepFace

objs = DeepFace.analyze(
  img_path = "test_images/img1.jpg", 
  actions = ['age', 'gender', 'race', 'emotion'],
)

Action: emotion: 100%|██████████| 4/4 [00:00<00:00, 12.13it/s]


In [2]:
import cv2
from cv2.data import haarcascades
from deepface.DeepFace import analyze

camera = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(haarcascades + "haarcascade_frontalface_default.xml")


while True:

    ret, frame = camera.read()

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Convert grayscale frame to RGB format
    rgb_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    largest_face = max(faces, key=lambda x: x[2] * x[3], default=None)

    if largest_face is not None:
        x, y, w, h = largest_face
        face_roi = rgb_frame[y : y + h, x : x + w]

        result = analyze(face_roi, actions=["emotion"], enforce_detection=False)

        # Determine the dominant emotion
        emotion = result[0]["dominant_emotion"]
        print(result[0]["face_confidence"])

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        # Define the colors for each emotion
        emotion_colors = {
            'angry': (0, 0, 255),
            'disgust': (0, 255, 0),
            'fear': (255, 0, 0),
            'happy': (255, 255, 0),
            'sad': (0, 255, 255),
            'surprise': (255, 0, 255),
            'neutral': (128, 128, 128)
        }

        # Define the position and size of the emotion bars
        bar_x = 30
        bar_y = 20
        bar_width = 150
        bar_height = 20
        bar_distance = 10

        emotions: dict[str, float] = result[0]["emotion"]
        i = 0
        # Draw the emotion bars
        for emotion, probability in emotions.items():
            # Calculate the height of the bar based on the probability
            bar_length = int(probability / 100 * bar_width)
            
            bar_offset = i * (bar_height + bar_distance)

            # Calculate the position of the top-left corner of the bar
            bar_top_left = (bar_x, bar_y + bar_offset)

            # Calculate the position of the bottom-right corner of the bar
            bar_bottom_right = (bar_x + bar_length, bar_y + bar_height + bar_offset)

            # Draw the bar
            cv2.rectangle(frame, bar_top_left, bar_bottom_right, emotion_colors[emotion], -1)

            # Add the emotion label
            cv2.putText(frame, emotion, (bar_x + bar_width + 30, bar_bottom_right[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
            i += 1

    # for result in results:
    #     # Extract the face ROI (Region of Interest)
    #     # face_roi = rgb_frame[y : y + h, x : x + w]

    #     # Perform emotion analysis on the face ROI
    #     # result = analyze(face_roi, actions=["emotion"], enforce_detection=False)
    #     x,y,w,h,_,_ = result["region"].values()

    #     # Determine the dominant emotion
    #     emotion = result["dominant_emotion"]

    #     # Draw rectangle around face and label with predicted emotion
    #     cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
    #     cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)


    # Display the resulting frame
    cv2.imshow("Real-time Emotion Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.96
0.93
0.92
0.92
0.94
0.94
0.94
0.94
0.94
0.91
0.93
0.92
0.92
0.92
0.92
0.91
0.92
0.91
0.92
0.94
0.95
0.96
0.95
0.94
0.93
0
0.93
0.93
0.94
0.95
0.94
0.93
0.94
0.96
0.96
0.97
0
0
0
0
0
0
0
0
0
0
0
0.99
0.97
0.98
0.98
0.99
0.98
0.96
0.95
0.97
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.98
0.98
0.96
0.98
0.96
0.96
0.95
0.94
0.93
0.92
0.92
0.93
0.94
0.94
0.95
0.96
0.93
0.94
0.95
0.92
0.93
0.94
0.92
0.91
0.91
0.94
0.94
0.94
0.94
0.94
0.96
0.96
0.96
0.96
0.96
0.95
0.95
0.92
0.94
0.94
0.95
0.94
0.96
0.95
0.94
0.93
0.93
0.95
0.94
0.95
0.95
0.95
0.98
0.96
0
0
0
0
0.99
0.95
0.95
0.96
0.96
0.98
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.96
0
0.96
0
0.98
0.96
0
0
0.96
0
0.97
0.97
0.96
0.97
0
0
0.98
0.93
0.97
0.95
0
0
0.97
0.97
0.93
0.95
0.97
0.96
0.98
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.99
0.99
0.98
0.99
0.99
0.99
0
0.98
0
0.98
0.97
1.0
0.99
0.99
0.98
0.99
0.9

KeyboardInterrupt: 

: 